In [41]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt
from datetime import datetime
import folium
import folium.plugins as plugins
from sklearn.cluster import KMeans
import seaborn as sns; sns.set()
import geopy.distance
import math
from pandas import option_context
import json
from collections import Counter

In [316]:
data = pd.read_csv("./data/pickup_delivery_occurences_every_hour.csv", index_col=0)

In [317]:
data.head()

,Zone,Period,Pickup occurences,Delivery occurences
0,1,0,"{""0"": 362, ""1"": 3}","{""0"": 362, ""1"": 3}"
1,1,1,"{""0"": 362, ""1"": 3}","{""0"": 360, ""1"": 4, ""2"": 1}"
2,1,2,"{""0"": 363, ""1"": 2}","{""0"": 361, ""1"": 3, ""2"": 1}"
3,1,3,"{""0"": 364, ""1"": 1}","{""0"": 364, ""1"": 1}"
4,1,4,"{""0"": 365}","{""0"": 364, ""1"": 1}"


In [111]:
with option_context('display.max_colwidth', 400):
    display(data.loc[(data['Zone'] == 126)])

,Zone,Period,Pickup occurences,Delivery occurences,Pickup distribution
375,126,Morning,"{""0"": 152, ""1"": 104, ""2"": 66, ""3"": 26, ""4"": 15, ""5"": 2}","{""0"": 258, ""1"": 85, ""2"": 20, ""3"": 1, ""4"": 1}",NaN
376,126,Day,"{""0"": 143, ""1"": 118, ""2"": 56, ""3"": 30, ""4"": 9, ""5"": 8, ""6"": 1}","{""0"": 134, ""1"": 123, ""2"": 63, ""3"": 24, ""4"": 14, ""5"": 5, ""6"": 1, ""7"": 1}",NaN
377,126,Evening,"{""0"": 57, ""1"": 77, ""2"": 82, ""3"": 69, ""4"": 32, ""5"": 24, ""6"": 18, ""7"": 6}","{""0"": 35, ""1"": 45, ""2"": 69, ""3"": 68, ""4"": 58, ""5"": 48, ""6"": 15, ""7"": 16, ""8"": 6, ""9"": 2, ""10"": 1, ""11"": 2}",NaN


In [318]:
data['Pickup distribution'] = np.nan
data['Max value deliveries'] = np.nan
data['Pickup distribution length match'] = np.nan
len(data)

6096

In [319]:
data = pd.read_csv("./data/pickup_delivery_occurences_every_hour.csv", index_col=0)

data['Pickup distribution'] = np.nan
data['Delivery distribution'] = np.nan

# pickups
for i in range(0,6096):
    
    d = json.loads(data.iloc[i,2])
    keys = list(range(int(max(d, key=int))+1))
    change = False
    for key in keys:
        if (str(key) not in d):
            change = True
            d[str(key)] = 0
    json_d = json.dumps(d)
    if change:
        data.iloc[i,2] = str(json_d)
        
        out_list = [round(d[str(k)]/365,3) for k in keys]
        data.iloc[i,4] = str(out_list)
    
    else:
        out_list = [round(d[str(k)]/365, 3) for k in keys]
        data.iloc[i,4] = str(out_list)

        
# deliveries
for i in range(0,762):
    
    d = json.loads(data.iloc[i,3])
    keys = list(range(int(max(d, key=int))+1))
    change = False
    for key in keys:
        if (str(key) not in d):
            change = True
            d[str(key)] = 0
    json_d = json.dumps(d)
    if change:
        data.iloc[i,3] = str(json_d)
        
        out_list = [round(d[str(k)]/365,3) for k in keys]
        data.iloc[i,5] = str(out_list)
    # Fill pickup distribution
    else:
        out_list = [round(d[str(k)]/365, 3) for k in keys]
        data.iloc[i,5] = str(out_list)
    
    

In [324]:
with option_context('display.max_colwidth', 400):
    display(data.head(350))


,Zone,Period,Pickup occurences,Delivery occurences,Pickup distribution,Delivery distribution
0,1,0,"{""0"": 362, ""1"": 3}","{""0"": 362, ""1"": 3}","[0.992, 0.008]","[0.992, 0.008]"
1,1,1,"{""0"": 362, ""1"": 3}","{""0"": 360, ""1"": 4, ""2"": 1}","[0.992, 0.008]","[0.986, 0.011, 0.003]"
2,1,2,"{""0"": 363, ""1"": 2}","{""0"": 361, ""1"": 3, ""2"": 1}","[0.995, 0.005]","[0.989, 0.008, 0.003]"
3,1,3,"{""0"": 364, ""1"": 1}","{""0"": 364, ""1"": 1}","[0.997, 0.003]","[0.997, 0.003]"
4,1,4,"{""0"": 365}","{""0"": 364, ""1"": 1}",[1.0],"[0.997, 0.003]"
...,...,...,...,...,...,...
345,15,9,"{""0"": 318, ""1"": 44, ""2"": 3}","{""0"": 323, ""1"": 41, ""2"": 1}","[0.871, 0.121, 0.008]","[0.885, 0.112, 0.003]"
346,15,10,"{""0"": 336, ""1"": 25, ""2"": 3, ""3"": 1}","{""0"": 333, ""1"": 29, ""2"": 3}","[0.921, 0.068, 0.008, 0.003]","[0.912, 0.079, 0.008]"
347,15,11,"{""0"": 329, ""1"": 33, ""2"": 3}","{""0"": 344, ""1"": 19, ""2"": 2}","[0.901, 0.09, 0.008]","[0.942, 0.052, 0.005]"
348,15,12,"{""0"": 335, ""1"": 28, ""2"": 2}","{""0"": 337, ""1"": 26, ""2"": 1, ""4"": 1, ""3"": 0}","[0.918, 0.077, 0.005]","[0.923, 0.071, 0.003, 0.0, 0.003]"


In [325]:
data.to_csv("./data/pickup_delivery_distributions_ever_hour.csv")